# naver api 이용해 감성분석 진행 

In [6]:
import warnings
warnings.filterwarnings('ignore')

## 00. 연습

In [11]:
import sys
import requests
import json

client_id = " ## "
client_secret = " ## "
url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

content = "싸늘하다. 가슴에 비수가 날아와 꽂힌다."
data = {
  "content": content
}

print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

{
    "content": "\uc2f8\ub298\ud558\ub2e4. \uac00\uc2b4\uc5d0 \ube44\uc218\uac00 \ub0a0\uc544\uc640 \uaf42\ud78c\ub2e4."
}
{"document":{"sentiment":"negative","confidence":{"negative":99.3676,"positive":0.0077638607,"neutral":0.62463653}},"sentences":[{"content":"싸늘하다.","offset":0,"length":5,"sentiment":"negative","confidence":{"negative":0.9961359,"positive":2.2740211E-4,"neutral":0.0036366575},"highlights":[{"offset":0,"length":4}]},{"content":" 가슴에 비수가 날아와 꽂힌다.","offset":5,"length":17,"sentiment":"negative","confidence":{"negative":0.927976,"positive":7.042612E-4,"neutral":0.07131973},"highlights":[{"offset":1,"length":15}]}]}


## 01. 데이터 불러오기

In [109]:
import pandas as pd
data = pd.read_excel('train_data/coupang_naverapi_data.xlsx',header=None)

In [110]:
data # 3960개의 데이터

,0,1
0,0,2021년 2월 15일 최근 도정입니다
1,1,2월 26일 배송 받았고 처음 밥해 먹었습니다
2,2,평들이 좋길래 믿고 구매해봤습니다
3,3,만 너무 실망입니다
4,4,우선 쌀알들이 금가있고 쌀눈이 깨져있고 누렇고 난리도 아닙니다
...,...,...
3955,3955,"둘째 아들이 흰 쌀밥만 고집해서 평소 밥으로 해주던\n김밥이나 주먹밥, 유부초밥으로..."
3956,3956,가격대비 쌀이 괜찮아서 재구매 했습니다
3957,3957,처음 샀을때보다 가격이 좀 올랐네요
3958,3958,잡곡을 많이 섞어 밥을 했는데쌀로만 밥을\n지어도 쌀이 좋은 편입니다


네이버에서 1000자씩 1000번까지만 무료이기 때문에 1000자 이내로 문장을 합쳐서 한번에 분석할 것임 -> 
4문장합치면 가능

In [ ]:
# 문장 합치기
qqq =''
for i in range(1, len(data)+1 ):
    if i %4 == 0 : #4개씩 합치는 것으로 지정함 : 개인이 회당 천자 넘지 않도록 설정
        ppp = '||-||'
    else:
        ppp = ' '
    qqq = qqq + data[1][i-1] + ppp    

# 4개 단위 분리
p = qqq.split('||-||')
len(p) # 991세트 생성

## 02. api이용 감성 파악 

In [ ]:
sentence, sentiment, = [],[]

# content 지정
for n in range(len(p)): # 감성분석할 데이터 세트 
    content = p[n] # 세트 넣기

    # 딕셔너리 생성
    data = { "content": content }

    # print(json.dumps(data, indent=4, sort_keys=True))
    result = json.loads(str(requests.post(url, data=json.dumps(data), headers=headers).text))

    for i in range( len(result['sentences']) ):
          sentence.append( result['sentences'][i]['content'] )
          sentiment.append( result['sentences'][i]['sentiment'] )


In [112]:
total_result = pd.DataFrame({'문장':sentence, '감성':sentiment })
total_result

,문장,감성
0,2021년 2월 15일 최근 도정입니다,neutral
1,2월 26일 배송 받았고 처음 밥해 먹었습니다,neutral
2,평들이 좋길래 믿고 구매해봤습니다,neutral
3,만 너무 실망입니다,negative
4,우선 쌀알들이 금가있고 쌀눈이 깨져있고 누렇고 난리도 아닙니다,negative
...,...,...
4728,"둘째 아들이 흰 쌀밥만 고집해서 평소 밥으로 해주던\n김밥이나 주먹밥, 유부초밥으...",neutral
4729,가격대비 쌀이 괜찮아서 재구매 했습니다,positive
4730,처음 샀을때보다 가격이 좀 올랐네요,negative
4731,잡곡을 많이 섞어 밥을 했는데쌀로만 밥을\n지어도 쌀이 좋은 편입니다,positive


In [119]:
# 엑셀로 저장
total_result.to_excel('naver_senti_sentence_1part.xlsx',encoding='utf-8')